In [1]:
# display cells to maximum width 
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>.output_result { max-width:100% !important; }</style>"))

# lets you preint multiple outputs per cell, not just last
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import pandas as pd
import requests
import bs4
from bs4 import BeautifulSoup
import re
import collections
import os
import json
import numpy as np
import pickle
from functools import reduce
import time
from time import sleep
import concurrent
import multiprocessing
import datetime as dt
from datetime import date
import pathlib
import configparser
import sys
import urllib
import zipfile
import csv
sys.path.insert(0, '/Volumes/TOSHIBA_EXT/ISB/clinical_trials/pymetamap-master')
from pymetamap import MetaMap  # https://github.com/AnthonyMRios/pymetamap/blob/master/pymetamap/SubprocessBackend.py
from pandas import ExcelWriter
import ast
import glob
from tqdm import tqdm
import subprocess
import shlex


In [3]:
# %pip install thefuzz
# %pip install levenshtein
# %pip install xlsxwriter

from thefuzz import fuzz # fuzzy matching explained: https://www.datacamp.com/tutorial/fuzzy-string-python

In [4]:
def convert_seconds_to_hms(seconds):
    """ converts the elapsed time or runtime to hours, min, sec """
    hours = seconds // 3600
    seconds %= 3600
    minutes = seconds // 60
    seconds %= 60
    return hours, minutes, seconds

In [6]:
def get_token_sort_ratio(str1, str2):
    """ fuzzy matching explained: https://www.datacamp.com/tutorial/fuzzy-string-python """

    try:
        return fuzz.token_sort_ratio(str1, str2)
    except:
        return None
sort_ratio = np.vectorize(get_token_sort_ratio)

def get_token_set_ratio(str1, str2):
    """ fuzzy matching explained: https://www.datacamp.com/tutorial/fuzzy-string-python """
    try:
        return fuzz.token_set_ratio(str1, str2)
    except:
        return None  
set_ratio = np.vectorize(get_token_set_ratio)

def get_similarity_score(str1, str2):
    """ fuzzy matching explained: https://www.datacamp.com/tutorial/fuzzy-string-python """
    try:
        return fuzz.ratio(str1, str2)
    except:
        return None
sim_score = np.vectorize(get_similarity_score)

In [7]:
def get_raw_ct_data():
    term_program_flag = True
    global data_dir
    global data_extracted

    # get all the links and associated dates of upload into a dict called date_link
    url_all = "https://aact.ctti-clinicaltrials.org/download"
    response = requests.get(url_all)
    soup = BeautifulSoup(response.text)
    body = soup.find_all('option') #Find all
    date_link = {}
    for el in body:
        tags = el.find('a')
        try:
            zip_name = tags.contents[0].split()[0]
            date = zip_name.split("_")[0]
            date = dt.datetime.strptime(date, '%Y%m%d').date()
            date_link[date] = tags.get('href')
        except:
            pass
    latest_file_date = max(date_link.keys())   # get the date of the latest upload
    url = date_link[latest_file_date]   # get the corresponding download link of the latest upload so we can download the raw data
    date_string = latest_file_date.strftime("%m_%d_%Y")
    data_dir = "{}/data".format(pathlib.Path.cwd())
    data_extracted = data_dir + "/{}_extracted".format(date_string)
    data_path = "{}/{}_pipe-delimited-export.zip".format(data_dir, date_string)

    if not os.path.exists(data_path):   # if folder containing most recent data doesn't exist, download and extract it into data folder

        term_program_flag = False   # flag below for terminating program if latest download exists (KG is assumed up to date)
        print("Attempting download of Clinical Trial data as of {}".format(date_string))
        try:
            response = requests.get(url)
            if response.status_code == 200:
                with open(data_path, 'wb') as file:
                    file.write(response.content)
                print("Finished download of zip")
                with zipfile.ZipFile(data_path, 'r') as download:
                    print("Unzipping data")
                    download.extractall(data_extracted)
        except:
            print("Failed to scrape AACT for download. Please navigate to https://aact.ctti-clinicaltrials.org/download and manually download zip file.")
            print("Please store the downloaded zip in the /data directory")
            done = input("Type Done when done: ")
            if done == "Done":
                data_dir = "{}/data".format(pathlib.Path.cwd())
                list_of_files = glob.glob(data_dir + "/*") # get all files in directory
                try:
                    latest_file = max(list_of_files, key=os.path.getctime) # get the most recent file in the directory
                    print("File found at: ")
                    print(latest_file)
                    print("Please make sure this the correct zip file from AACT")
                    try:
                        with zipfile.ZipFile(latest_file, 'r') as download:
                            print("Unzipping data into")
                            cttime = os.path.getctime(latest_file)
                            date_string = dt.datetime.fromtimestamp(cttime).strftime('%m_%d_%Y')
                            data_extracted = data_dir + "/{}_extracted".format(date_string)
                            print(data_extracted)
                            download.extractall(data_extracted)
                    except:
                        print("Assuming data is already unzipped")
                except:
                    print("Unable to download and extract Clincal Trial data.")
                    print("Cannot find pipe-delimited zip in /data folder.")
    else:
        print("KG is already up to date.")

    return {"term_program_flag": term_program_flag, "data_extracted_path": data_extracted, "date_string": date_string}


In [8]:
def read_raw_ct_data(flag_and_path, subset_size):
    if flag_and_path["term_program_flag"]:
        print("Exiting program. Assuming KG has already been constructed from most recent data dump from AACT.")
        exit()
    else:
        data_extracted = flag_and_path["data_extracted_path"]
        
        # read in pipe-delimited files 
        conditions_df = pd.read_csv(data_extracted + '/conditions.txt', sep='|', index_col=False, header=0)
        interventions_df = pd.read_csv(data_extracted + '/interventions.txt', sep='|', index_col=False, header=0)
        alternate_interventions_df = pd.read_csv(data_extracted + '/intervention_other_names.txt', sep='|', index_col=False, header=0)

        if subset_size:   # if a subset size is given, we are running this script on a small subset of the dataset
            conditions_df = conditions_df.sample(n=subset_size)
            interventions_df = interventions_df.sample(n=subset_size)
            alternate_interventions_df = alternate_interventions_df.sample(n=subset_size)

    return {"conditions": conditions_df, "interventions": interventions_df, "interventions_alts": alternate_interventions_df}



In [9]:
def de_ascii_er(text):
    non_ascii = "[^\x00-\x7F]"
    pattern = re.compile(r"[^\x00-\x7F]")
    non_ascii_text = re.sub(pattern, ' ', text)
    return non_ascii_text

In [10]:
def start_metamap_servers(metamap_dirs):
    global metamap_pos_server_dir
    global metamap_wsd_server_dir
    metamap_pos_server_dir = 'bin/skrmedpostctl' # Part of speech tagger
    metamap_wsd_server_dir = 'bin/wsdserverctl' # Word sense disambiguation 
    
    metamap_executable_path_pos = os.path.join(metamap_dirs['metamap_base_dir'], metamap_pos_server_dir)
    metamap_executable_path_wsd = os.path.join(metamap_dirs['metamap_base_dir'], metamap_wsd_server_dir)
    command_pos = [metamap_executable_path_pos, 'start']
    command_wsd = [metamap_executable_path_wsd, 'start']

    # Start servers, with open portion redirects output of metamap server printing output to NULL
    with open(os.devnull, "w") as fnull:
        result_post = subprocess.call(command_pos, stdout = fnull, stderr = fnull)
        result_wsd = subprocess.call(command_wsd, stdout = fnull, stderr = fnull)
    sleep(5)

def stop_metamap_servers(metamap_dirs):
    metamap_executable_path_pos = os.path.join(metamap_dirs['metamap_base_dir'], metamap_pos_server_dir)
    metamap_executable_path_wsd = os.path.join(metamap_dirs['metamap_base_dir'], metamap_wsd_server_dir)
    command_pos = [metamap_executable_path_pos, 'stop']
    command_wsd = [metamap_executable_path_wsd, 'stop']
    
    # Stop servers, with open portion redirects output of metamap server printing output to NULL
    with open(os.devnull, "w") as fnull:
        result_post = subprocess.call(command_pos, stdout = fnull, stderr = fnull)
        result_wsd = subprocess.call(command_wsd, stdout = fnull, stderr = fnull)
    sleep(2)


In [ ]:
# # Assuming dirs['mm_base_dir'], mm_pos_server_dir, and other variables are defined
# executable_path = os.path.join(dirs['mm_base_dir'], mm_pos_server_dir)
# command = [executable_path, 'start']

# with open(os.devnull, 'w') as fnull:
#     result = subprocess.call(command, stdout=fnull, stderr=fnull)

In [ ]:
# metamap_base_dir = '/Volumes/TOSHIBA_EXT/ISB/clinical_trials/public_mm/' # for running on local
# metamap_bin_dir = 'bin/metamap18'

# metamap_pos_server_dir = 'bin/skrmedpostctl' # Part of speech tagger
# metamap_wsd_server_dir = 'bin/wsdserverctl' # Word sense disambiguation 

# # os.system(metamap_base_dir + metamap_pos_server_dir + ' start') # Part of speech tagger
# # os.system(metamap_base_dir + metamap_wsd_server_dir + ' start') # Word sense disambiguation 

# os.system(metamap_base_dir + metamap_pos_server_dir + ' stop') # Part of speech tagger
# os.system(metamap_base_dir + metamap_wsd_server_dir + ' stop') # Word sense disambiguation 



In [11]:
def check_os():
    if "linux" in sys.platform:
        print("Linux platform detected")
        metamap_base_dir = "{}/metamap/".format(pathlib.Path.cwd().parents[0])
        metamap_bin_dir = 'bin/metamap20'
    else:
        metamap_base_dir = '/Volumes/TOSHIBA_EXT/ISB/clinical_trials/public_mm/' # for running on local
        metamap_bin_dir = 'bin/metamap18'
        
    return {"metamap_base_dir":metamap_base_dir, "metamap_bin_dir":metamap_bin_dir}
        

In [12]:
def run_metamap(input_term, params, mm, cond_or_inter, csv_writer):
    from_metamap = []
    if params.get("exclude_sts") is None: # exclude_sts is used for Interventions. restrict_to_sts is used for Conditions. So, the logic is, if we're mapping Conditions, execute "if" part of code. If we're mapping Interventions, execute "else" part of code
        try:
            concepts,error = mm.extract_concepts([input_term],
                                                 restrict_to_sts = params["restrict_to_sts"],
                                                 term_processing = params["term_processing"],
                                                 ignore_word_order = params["ignore_word_order"],
                                                 strict_model = params["strict_model"],
                                                )
            for concept in concepts:
                concept_info = []
                concept = concept._asdict()
                concept_info.extend([cond_or_inter,input_term])
                concept_info.extend([concept.get(k) for k in ['preferred_name', 'cui', 'score', 'semtypes']])
                from_metamap.append(concept_info)
        except:
            from_metamap.extend([input_term, None, None, None, None, None, None])
    else:
        try:
            concepts,error = mm.extract_concepts([input_term],
                                                 exclude_sts = params["exclude_sts"],
                                                 term_processing = params["term_processing"],
                                                 ignore_word_order = params["ignore_word_order"],
                                                 strict_model = params["strict_model"],
                                                )
            for concept in concepts:
                concept_info = []
                concept = concept._asdict()
                concept_info.extend([cond_or_inter,input_term])
                concept_info.extend([concept.get(k) for k in ['preferred_name', 'cui', 'score', 'semtypes']])
                from_metamap.append(concept_info)
        except:
            from_metamap.extend([input_term, None, None, None, None, None, None])
        
    for result in from_metamap:
        # print(result)
        csv_writer.writerow(result)
    return from_metamap

In [13]:
def parallelize_metamap(term_list, params, cond_or_inter, flag_and_path, csv_writer):
    LENGTH = len(term_list)  # Number of iterations required to fill progress bar (pbar)
    pbar = tqdm(total=LENGTH, desc="% {}s mapped".format(cond_or_inter), position=0, leave=True, mininterval = LENGTH/10)  # Init progress bar

    start_metamap_servers(metamap_dirs) # start the MetaMap servers
    mm = MetaMap.get_instance(metamap_dirs["metamap_base_dir"] + metamap_dirs["metamap_bin_dir"])
    with concurrent.futures.ThreadPoolExecutor((multiprocessing.cpu_count()*2) - 1) as executor:
        futures = [executor.submit(run_metamap, term, params, mm, cond_or_inter, csv_writer) for term in term_list]
        for _ in concurrent.futures.as_completed(futures):
            pbar.update(n=1)  # Increments counter
    stop_metamap_servers(metamap_dirs) # stop the MetaMap servers
    

In [ ]:
# import os
# import subprocess

# command = ["executable", "argument_1", "argument_2"]

# with open(os.devnull, "w") as fnull:
#     result = subprocess.call(command, stdout = fnull, stderr = fnull)

In [ ]:
# from concurrent.futures import ThreadPoolExecutor, as_completed

# #   ... snip.....

# LENGTH = len(lista_hosts)  # Number of iterations required to fill pbar
# pbar = tqdm(total=LENGTH, desc='consulta_comando', position=0, leave=True)  # Init pbar
# with ThreadPoolExecutor(200) as executor:
#     futures = [executor.submit(comando_remoto.comando_remoto_hosts,
#                                add_dns_cc.add_dns_concesion(host),
#                                comando, res_versiones, user, clave_rsa,
#                                passwd) for host in lista_hosts]
#     for _ in as_completed(futures):
#         pbar.update(n=1)  # Increments counter

# USE METAMAP LOCAL TO MAP REMAINING TERMS

In [22]:
def term_list_to_mm(df_dict, flag_and_path):
        
    metamap_version = [int(s) for s in re.findall(r'\d+', metamap_dirs.get('metamap_bin_dir'))] # get MetaMap version being run 
    # some input terms have () with additional text, like an abbreviation, in them. split them out to facilitate better mapping using these regex patterns that we use to find substrings inside and outside ()
    pattern_outside = r'(?<=\().+?(?=\))|([^(]+)'
    pattern_inside = r'\(([^)]+)\)'
    relevant_date = flag_and_path["date_string"]   # get date of bulk download of clinical trial data
    deasciier = np.vectorize(de_ascii_er) # vectorize function

    # -------    CONDITIONS    ------- #
    print("Using UMLS MetaMap to get mappings for CONDITIONS. MetaMap returns mappings, CUIs, and semantic type of mapping.")
    conditions = df_dict["conditions"][['id', 'nct_id', 'downcase_name']]
    conditions = conditions.copy()
    conditions.rename(columns = {'downcase_name':'orig_con'}, inplace = True)

    if metamap_version[0] >= 20:
        matches_outside = conditions['orig_con'].str.extract(pattern_outside)
        conditions['orig_con_outside'] = matches_outside[0].fillna('')
        matches_inside = conditions['orig_con'].str.extract(pattern_inside)
        conditions['orig_con_inside'] = matches_inside[0].fillna('')

    else:
        conditions['deascii_con'] = deasciier(conditions['orig_con'])
        matches_outside = conditions['deascii_con'].str.extract(pattern_outside)
        conditions['deascii_con_outside'] = matches_outside[0].fillna('')
        matches_inside = conditions['deascii_con'].str.extract(pattern_inside)
        conditions['deascii_con_inside'] = matches_inside[0].fillna('')
    
#     see MetaMap Usage instructions: https://lhncbc.nlm.nih.gov/ii/tools/MetaMap/Docs/MM_2016_Usage.pdf
#     condition_args = ['--sldi -I -C -J acab,anab,cgab,comd,dsyn,inpo,mobd,neop,patf -z -i -f']  # see https://lhncbc.nlm.nih.gov/ii/tools/MetaMap/Docs/SemanticTypes_2018AB.txt for semantic types ("acab,anab,etc.")
    condition_semantic_type_restriction = ['acab,anab,cgab,comd,dsyn,inpo,mobd,neop,patf,clna,fndg']  # see https://lhncbc.nlm.nih.gov/ii/tools/MetaMap/Docs/SemanticTypes_2018AB.txt for semantic types ("acab,anab,etc.")
    params = {"restrict_to_sts":condition_semantic_type_restriction, "term_processing":True, "ignore_word_order":True, "strict_model":False} # strict_model and relaxed_model are presumably opposites? relaxed_model = True is what I want, but that option appears to be broken in Pymetamap (returns no results when used). Using strict_model = False instead...
    
    # prep output file of Metamap results
    filename = f"{relevant_date}_metamap_output.tsv"
    metamap_output = open(filename, 'w+', newline='')
    col_names = ['term_type', 'clin_trial_term','metamap_preferred_name', 'metamap_cui', 'metamap_score', 'metamap_semantic_type']
    csv_writer = csv.writer(metamap_output, delimiter='\t')
    csv_writer.writerow(col_names)
    
    if metamap_version[0] >= 20:
        print("MetaMap version >= 2020, conduct mapping on original terms")
        orig_cons = conditions.orig_con.unique().tolist()
        orig_cons = list(filter(None, orig_cons))
        orig_cons = [str(i) for i in orig_cons]
        parallelize_metamap(orig_con, params, "condition", flag_and_path, csv_writer)
    else:
        print("MetaMap version < 2020, conduct mapping on terms after removing ascii characters")
        deascii_cons = conditions.deascii_con.unique().tolist()
        deascii_cons = list(filter(None, deascii_cons))
        deascii_cons = [str(i) for i in deascii_cons]
        parallelize_metamap(deascii_cons, params, "condition", flag_and_path, csv_writer)
        
        """ If the substring that was either outside or inside the () is identical to the term from which it came from, or actually any of the columns have the same value, put None in that cell/put None where that term is duplicated """    
    # Iterate through each column in the DataFrame
    for col1 in conditions.columns:
        for col2 in conditions.columns:
            # Skip comparing a column with itself
            if col1 != col2:
                # Check if the values in col2 are duplicates of col1
                conditions[col2] = conditions.apply(lambda row: row[col2] if row[col2] != row[col1] else None, axis=1)
    # Drop duplicate columns (keeping the first instance)
    conditions = conditions.T.drop_duplicates().T

    conditions.to_csv('{}_conditions.tsv'.format(relevant_date), sep="\t", index=False, header=True) # output interventions to TSV
    
    # -------    INTERVENTIONS    ------- #
    print("Using UMLS MetaMap to get mappings for INTERVENTIONS. MetaMap returns mappings, CUIs, and semantic type of mapping.")
    
    """ Interventions requires unique handling. Another table gives possible alternate names for the interventions in addition to the "original" names. 
        We may map on the alternate names column
        We take the interventions, take the ascii and deasciied versions of them,
        and split substrings in parentheses out of them. We perform MetaMapping on the
        original term or the deasciied term dependinging on what operating system we
        are on. If the mapped term passes the fuzzy scoring thesholds for any of the
        terms (original, deasciied, original inside the parentheses, deasciied inside
        the parentheses, original outside the parentheses, deasciied outside the
        parentheses""" 

    interventions_df = df_dict["interventions"]
    interventions_df['orig_downcase_name'] = interventions_df['name'].str.lower()
    interventions_alts = df_dict["interventions_alts"]
    interventions_alts['alt_downcase_name'] = interventions_alts['name'].str.lower()
    
    orig_ints = interventions_df["orig_downcase_name"]
    orig_ints = list(orig_ints.unique())
    orig_ints = list(filter(None, orig_ints))
    alt_ints = interventions_alts["alt_downcase_name"]
    alt_ints = list(alt_ints.unique())
    alt_ints = list(filter(None, alt_ints))

    params = {"exclude_sts":condition_semantic_type_restriction, "term_processing":True, "ignore_word_order":True, "strict_model":False} # strict_model and relaxed_model are presumably opposites? relaxed_model = True is what I want, but that option appears to be broken in Pymetamap (returns no results when used). Using strict_model = False instead...
    """ Send the prepared interventions to MetaMap now. If we are on OSX, we have to use MetaMap 2018, which requires deasciied terms. If we are on Linux, we can use MetaMap 2020, which does not require such preprocessing """
    if metamap_version[0] < 20:
        deasciier = np.vectorize(de_ascii_er) # vectorize function
        #  -------   original interventions  -------- #
        orig_ints = [str(i) for i in orig_ints]
        orig_ints = deasciier(orig_ints) # perform deascii-ing on original intervention names
        orig_ints = list(orig_ints)
        print("MetaMap version < 2020, conduct mapping on original interventions after removing ascii characters")
        parallelize_metamap(orig_ints, params, "intervention", flag_and_path, csv_writer)
        #  ---------   alternate interventions ------- #
        alt_ints = [str(i) for i in alt_ints]
        alt_ints = deasciier(alt_ints) # perform deascii-ing on alternate intervention names
        alt_ints = list(alt_ints)
        parallelize_metamap(alt_ints, params, "alternate_intervention", flag_and_path, csv_writer)

    else:

        #  -------   original interventions  -------- #
        print("MetaMap version >= 2020, conduct mapping on original interventions")
        parallelize_metamap(orig_ints, params, "intervention", flag_and_path, csv_writer)
        #  ---------   alternate interventions ------- #
        print("MetaMap version >= 2020, conduct mapping on alternate interventions")
        parallelize_metamap(alt_ints, params, "alternate_intervention", flag_and_path, csv_writer)

    interventions_all = pd.merge(interventions_df[["id", "nct_id", "intervention_type", "orig_downcase_name", "description"]], interventions_alts[["nct_id", "intervention_id", "alt_downcase_name"]], how='left', left_on=['id'], right_on = ['intervention_id'])
    interventions_all = interventions_all.astype(str)
    interventions_all = interventions_all.drop('nct_id_y', axis=1) # drop the redundant column now
    interventions_all.rename(columns = {'nct_id_x':'nct_id'}, inplace = True)

    interventions_all = interventions_all.sort_values(by='nct_id', ascending=False, na_position='last')
    interventions_all = interventions_all.drop('intervention_id', axis=1) # drop the redundant column now
    interventions_all.rename(columns = {'id':'intervention_id', 'orig_downcase_name':'orig_int', 'alt_downcase_name':'alt_int'}, inplace = True)

    if metamap_version[0] >= 20:
        matches_outside = interventions_all['orig_int'].str.extract(pattern_outside)
        interventions_all['orig_int_outside'] = matches_outside[0].fillna('')
        matches_inside = interventions_all['orig_int'].str.extract(pattern_inside)
        interventions_all['orig_int_inside'] = matches_inside[0].fillna('')

        matches_outside = interventions_all['alt_int'].str.extract(pattern_outside)
        interventions_all['alt_int_outside'] = matches_outside[0].fillna('')
        matches_inside = interventions_all['alt_in'].str.extract(pattern_inside)
        interventions_all['alt_int_inside'] = matches_inside[0].fillna('')
    else:

        interventions_all['deascii_orig_int'] = deasciier(interventions_all['orig_int'])
        interventions_all['deascii_alt_int'] = deasciier(interventions_all['alt_int'])

        matches_outside = interventions_all['deascii_orig_int'].str.extract(pattern_outside)
        interventions_all['deascii_orig_int_outside'] = matches_outside[0].fillna('')
        matches_inside = interventions_all['deascii_orig_int'].str.extract(pattern_inside)
        interventions_all['deascii_orig_int_inside'] = matches_inside[0].fillna('')

        matches_outside = interventions_all['deascii_alt_int'].str.extract(pattern_outside)
        interventions_all['deascii_alt_int_outside'] = matches_outside[0].fillna('')
        matches_inside = interventions_all['deascii_alt_int'].str.extract(pattern_inside)
        interventions_all['deascii_alt_name_inside'] = matches_inside[0].fillna('')

    """ I don't want to perform mapping on strings < 4 char in length; these are ambiguous and it's hard to make a call what that concept should be """
    """ Get character counts of all the columns to evaluate """    
    for col in interventions_all.columns: # get the char counts of each column
        char_count_col_name = col + '_char_count'
        interventions_all[char_count_col_name] = interventions_all[col].str.len()

    """ If char_count < 4, replace the string in the corresponding column with None so that we don't use it for comparison """    
    for col in interventions_all.columns[interventions_all.columns.str.contains("char_count")]:
        for index, value in interventions_all[col].items():
            if value < 4:
                # Find the column with the most similar name without "char_count" substring
                most_similar_col = interventions_all.columns[interventions_all.columns.str.replace("_char_count", "") == col.replace("_char_count", "")].values[0]
                # Update the value in the most similar column
                interventions_all.at[index, most_similar_col] = None
        interventions_all = interventions_all.drop(col, axis=1) # drop the count columns now  
        
    """ If the substring that was either outside or inside the () is identical to the term from which it came from, or actually any of the columns have the same value, put None in that cell/put None where that term is duplicated """    
    # Iterate through each column in the DataFrame
    for col1 in interventions_all.columns:
        for col2 in interventions_all.columns:
            # Skip comparing a column with itself
            if col1 != col2:
                # Check if the values in col2 are duplicates of col1
                interventions_all[col2] = interventions_all.apply(lambda row: row[col2] if row[col2] != row[col1] else None, axis=1)
    # Drop duplicate columns (keeping the first instance)
    interventions_all = interventions_all.T.drop_duplicates().T


    interventions_all.to_csv('{}_interventions.tsv'.format(relevant_date), sep="\t", index=False, header=True) # output interventions to TSV



In [15]:
def map_to_trial(df_dict, flag_and_path):
    # send mappings to interventions and conditions, group CUIs that correspond to input condition or intervention
    
    print("Mapping UMLS CURIEs and names back to clinical trials")
    relevant_date = flag_and_path["date_string"]   # get date of bulk download of clinical trial data
    metamap_version = [int(s) for s in re.findall(r'\d+', metamap_dirs.get('metamap_bin_dir'))] # get MetaMap version being run 

    metamap_input = "{}_metamap_output.tsv".format(relevant_date)
    metamapped = pd.read_csv(metamap_input, sep='\t', index_col=False, header=0)

    # get the full names of the semantic types so we know what we're looking at
    metamap_semantic_types = pd.read_csv("MetaMap_SemanticTypes_2018AB.txt")
    metamapped['metamap_semantic_type'] = metamapped['metamap_semantic_type'].str.replace(r'\[|\]', '', regex=True)
    sem_type_col_names = ["abbv", "group", "semantic_type_full"]
    metamap_semantic_types = pd.read_csv("MetaMap_SemanticTypes_2018AB.txt", sep="|", index_col=False, header=None, names=sem_type_col_names)
    sem_type_dict = dict(zip(metamap_semantic_types['abbv'], metamap_semantic_types['semantic_type_full'])) # make a dict of semantic type abbv and full name
    # Handle NaN (None) values in metamap_semantic_type column
    metamapped['metamap_semantic_type'] = metamapped['metamap_semantic_type'].apply(lambda x: x.split(',') if isinstance(x, str) else np.nan)
    # map semantic type abbreviations to the full name of the semantic type
    metamapped['metamap_semantic_type'] = metamapped['metamap_semantic_type'].apply(lambda x: '|'.join([sem_type_dict[term] if term in sem_type_dict else term for term in x]) if isinstance(x, list) else x)

    metamapped['metamap_preferred_name'] = metamapped['metamap_preferred_name'].str.lower()
    metamapped = metamapped.dropna(axis=0)
    metamapped = metamapped[["term_type", "clin_trial_term", "metamap_cui","metamap_preferred_name", "metamap_semantic_type"]]

    metamapped["metamap_term_info"] = metamapped[["metamap_cui", "metamap_preferred_name", "metamap_semantic_type"]].values.tolist() 
    metamapped.drop(["metamap_cui", "metamap_preferred_name", "metamap_semantic_type"], axis = 1, inplace = True)
    metamapped = metamapped.groupby(['term_type', 'clin_trial_term'])['metamap_term_info'].agg(list).reset_index()

    conditions = '{}_conditions.tsv'.format(relevant_date)
    conditions = pd.read_csv(conditions, sep='\t', index_col=False, header=0)
    interventions = '{}_interventions.tsv'.format(relevant_date)
    interventions = pd.read_csv(interventions, sep='\t', index_col=False, header=0)

    metamapped_con = metamapped.loc[metamapped['term_type'] == "condition"]
    metamapped_int = metamapped.loc[(metamapped['term_type'] == "intervention") | (metamapped['term_type'] == "alternate_intervention")]

    mapper_con = dict(zip(metamapped_con['clin_trial_term'], metamapped_con['metamap_term_info'])) # make a dict to map conditions
    mapper_int = dict(zip(metamapped_int['clin_trial_term'], metamapped_int['metamap_term_info'])) # make a dict to map interventions

#     cols_to_check = [ele for ele in conditions.columns if(ele not in ['id', 'nct_id', 'condition_id'])]
    cols_to_check = [ele for ele in conditions.columns if any([substr in ele for substr in ['_con']])]

    conditions["curie_info"] = None

    for index, row in conditions.iterrows():
        for col_name in cols_to_check:
            value = row[col_name]
            if value in mapper_con:
                curie_info = mapper_con[value]
                conditions.at[index, "curie_info"] = curie_info    
                
    conditions.to_csv('{}_conditions.tsv'.format(relevant_date), sep="\t", index=False, header=True) # output conditions to TSV

#     cols_to_check = [ele for ele in interventions.columns if(ele not in ['id', 'nct_id', 'intervention_id', 'intervention_type', 'description'])]
    cols_to_check = [ele for ele in interventions.columns if any([substr in ele for substr in ['_int']])]

    interventions["curie_info"] = None

    for index, row in interventions.iterrows():
        for col_name in cols_to_check:
            value = row[col_name]
            if value in mapper_int:
                curie_info = mapper_int[value]
                interventions.at[index, "curie_info"] = curie_info
    
    interventions.to_csv('{}_interventions.tsv'.format(relevant_date), sep="\t", index=False, header=True) # output interventions to TSV




In [16]:
def score_mappings(flag_and_path):
    
    print("Scoring mappings")
    relevant_date = flag_and_path["date_string"]   # get date of bulk download of clinical trial data
    
    #   -- --- --   CONDITIONS   -- --- -- #
    conditions = "{}_conditions.tsv".format(relevant_date)
    conditions = pd.read_csv(conditions, sep='\t', index_col=False, header=0)
    cols_to_check = [ele for ele in conditions.columns if any([substr in ele for substr in ['_con']])]
    conditions = conditions.where(pd.notnull(conditions), None)

    for index, row in conditions.iterrows():
        curies_sublists_scored = []
        for col_name in cols_to_check:
            value = row[col_name]
            curie_info = row["curie_info"]
            if None not in [value, curie_info]:
                curie_sublists = ast.literal_eval(curie_info)
                for sublist in curie_sublists:
                    sublist.append(f'sort_ratio: {get_token_sort_ratio(value, sublist[1])}')
                    sublist.append(f'similarity_score: {get_similarity_score(value, sublist[1])}')
                    curies_sublists_scored.append(sublist)
        conditions.at[index, "curie_info"] = curies_sublists_scored
    conditions.to_csv('{}_conditions.tsv'.format(relevant_date), sep="\t", index=False, header=True) # output to TSV

    #   -- --- --   INTERVENTIONS   -- --- -- #
    
    interventions = "{}_interventions.tsv".format(relevant_date)
    interventions = pd.read_csv(interventions, sep='\t', index_col=False, header=0)
    cols_to_check = [ele for ele in interventions.columns if any([substr in ele for substr in ['_int']])]
    interventions = interventions.where(pd.notnull(interventions), None)

    for index, row in interventions.iterrows():
        curies_sublists_scored = []
        for col_name in cols_to_check:
            value = row[col_name]
            curie_info = row["curie_info"]
            if None not in [value, curie_info]:
                curie_sublists = ast.literal_eval(curie_info)
                for sublist in curie_sublists:
                    sublist.append(f'sort_ratio: {get_token_sort_ratio(value, sublist[1])}')
                    sublist.append(f'similarity_score: {get_similarity_score(value, sublist[1])}')
                    curies_sublists_scored.append(sublist)

        interventions.at[index, "curie_info"] = curies_sublists_scored
    interventions.to_csv('{}_interventions.tsv'.format(relevant_date), sep="\t", index=False, header=True) # output interventions to TSV


In [17]:
def auto_select_curies(flag_and_path):
    print("Auto-selecting high scoring CURIEs")
    relevant_date = flag_and_path["date_string"]   # get date of bulk download of clinical trial data
    
    def filter_and_select_sublist(sublists):  # function to find the highest score of a CURIE, and pick that curie if it's greater than threshold of 88
        if sublists is None or len(sublists) == 0:
            return None

        high_score = -1
        selected_sublist = None

        sublists = ast.literal_eval(sublists)
        for sublist in sublists:

            if len(sublist) >= 4:
                sort_ratio = int(sublist[3].split(": ")[1])
                sim_score = int(sublist[4].split(": ")[1])
                max_score = max(sort_ratio, sim_score)
                if max_score > 88: 
                    if max_score > high_score:
                        high_score = max_score
                        selected_sublist = sublist
        return selected_sublist

    #   -----   -----    -----   -----   CONDITIONS   -----   -----    -----   -----  #

    conditions = "{}_conditions.tsv".format(relevant_date)
    conditions = pd.read_csv(conditions, sep='\t', index_col=False, header=0)
    """  Create an output TSV of CURIEs that are auto-selected based on passing the threshold of scoring > 88  """
    conditions['auto_selected_curie'] = conditions['curie_info'].apply(filter_and_select_sublist)  # select CURIE that scores highest using filter_and_select_sublist function = auto-select
    auto_selected_conditions = conditions[conditions[['auto_selected_curie']].notnull().all(1)]   # get the rows where a CURIE has been auto-selected
    auto_selected_conditions = auto_selected_conditions[["id", "nct_id", "orig_con", "curie_info", "auto_selected_curie"]]  # subset dataframe
    auto_selected_conditions.to_csv('{}_conditions_auto_selected.tsv'.format(relevant_date), sep="\t", index=False, header=True) # output to TSV

    conditions_manual_review = conditions[conditions["auto_selected_curie"].isna()]   # select rows where no CURIE was auto-selected
    conditions_manual_review = conditions_manual_review[["orig_con", "curie_info"]]  # subset

    """  Create an output TSV of possible CURIEs available for each term that was not auto-selected  """
    conditions_manual_review['curie_info'] = conditions_manual_review['curie_info'].apply(ast.literal_eval)   # in order to multi-index, we have to group-by the original input term. To do this, first convert the column to list of lists
    conditions_manual_review = conditions_manual_review.explode('curie_info')  # explode that column so every sublist is on a separate row
    conditions_manual_review['curie_info'] = conditions_manual_review['curie_info'].apply(lambda x: x[:3] if isinstance(x, list) else None)   # remove the scores (sort_ratio and similarity score) from the list, don't need them and they compromise readability of manual outputs 
    conditions_manual_review['curie_info'] = conditions_manual_review['curie_info'].apply(lambda x: ', '.join(x) if isinstance(x, list) else None)  # Multindexing does not work on lists, so remove the CURIE information out of the list to enable this

    conditions_manual_review['temp'] = "temp"   # create a temp column to facilitate multi-indexing
    conditions_manual_review.set_index(["orig_con", 'curie_info'], inplace=True)   # create index
    conditions_manual_review.drop(["temp"], axis = 1, inplace = True)   # drop the temp column
    conditions_manual_review['manually_selected_CURIE'] = None # make a column 

    conditions_manual_review.to_excel('{}_conditions_manual_review.xlsx'.format(relevant_date), engine='xlsxwriter', index=True)

    #   -----   -----    -----   -----   INTERVENTIONS   -----   -----    -----   -----  #
    interventions = "{}_interventions.tsv".format(relevant_date)
    interventions = pd.read_csv(interventions, sep='\t', index_col=False, header=0)
    """  Create an output TSV of CURIEs that are auto-selected based on passing the threshold of scoring > 88  """
    interventions['auto_selected_curie'] = interventions['curie_info'].apply(filter_and_select_sublist)
    auto_selected_interventions = interventions[interventions[['auto_selected_curie']].notnull().all(1)]
    auto_selected_interventions = auto_selected_interventions[["intervention_id", "nct_id", "intervention_type", "orig_int", "description", "curie_info", "auto_selected_curie"]]
    auto_selected_interventions.to_csv('{}_interventions_auto_selected.tsv'.format(relevant_date), sep="\t", index=False, header=True) # output interventions to TSV, avoid storing in memory

    interventions_manual_review = interventions[interventions["auto_selected_curie"].isna()]
    interventions_manual_review = interventions_manual_review[["intervention_type", "orig_int", "description", "curie_info"]]

    """  Create an output TSV of possible CURIEs available for each term that was not auto-selected  """
    interventions_manual_review['curie_info'] = interventions_manual_review['curie_info'].apply(ast.literal_eval)
    interventions_manual_review = interventions_manual_review.explode('curie_info')
    interventions_manual_review['curie_info'] = interventions_manual_review['curie_info'].apply(lambda x: x[:3] if isinstance(x, list) else None)   # remove the scores (sort_ratio and similarity score) from the list, don't need them and they compromise readability of manual outputs 
    interventions_manual_review['curie_info'] = interventions_manual_review['curie_info'].apply(lambda x: ', '.join(x) if isinstance(x, list) else None)

    interventions_manual_review['temp'] = "temp"
    interventions_manual_review.set_index(["intervention_type", "orig_int", "description", 'curie_info'], inplace=True)
    interventions_manual_review.drop(["temp"], axis = 1, inplace = True)
    interventions_manual_review['manually_selected_CURIE'] = None

    interventions_manual_review.to_excel('{}_interventions_manual_review.xlsx'.format(relevant_date), engine='xlsxwriter', index=True)

In [18]:
def run_ETL(subset_size):
    
    start_time_begin = time.time()
    flag_and_path = get_raw_ct_data() # download raw data
    end_time_download = time.time()
    elapsed_time = end_time_download - start_time_begin
    hours, minutes, seconds = convert_seconds_to_hms(elapsed_time)
    print(f"Approximate runtime for downloading or locating raw data: {hours} hours, {minutes} minutes, {seconds} seconds")
    
    global metamap_dirs
    metamap_dirs = check_os()
    df_dict = read_raw_ct_data(flag_and_path, subset_size) # read the clinical trial data
    
    # term_list_to_cache()
    
    start_time_mm = time.time()
    term_list_to_mm(df_dict, flag_and_path) # map using MetaMap
    end_time_mm = time.time()
    elapsed_time = end_time_mm - start_time_mm
    hours, minutes, seconds = convert_seconds_to_hms(elapsed_time)
    print(f"Approximate runtime for mapping: {hours} hours, {minutes} minutes, {seconds} seconds")
    
    map_to_trial(df_dict, flag_and_path) # map MetaMap terms back to trial 
    score_mappings(flag_and_path) # score the mappings
    auto_select_curies(flag_and_path) # select CURIEs automatically that pass score threshold
    end_time_end = time.time()
    elapsed_time = end_time_end - start_time_begin
    hours, minutes, seconds = convert_seconds_to_hms(elapsed_time)
    print(f"Approximate runtime for overall mapping: {hours} hours, {minutes} minutes, {seconds} seconds")

    

In [20]:
def test_or_prod():
    print("The test run of this code performs the construction of the KG on a random subset of 200 Conditions, 200 Interventions, and 200 Alternate Interventions from Clinical Trials.\n")
    test_or_prod = input("Is this a test run or the production of a new version of the KG? Enter Test for test, or Prod for production: ")
    if test_or_prod == "Test":
        subset_size = 300
        run_ETL(subset_size)
    elif test_or_prod == "Prod":
        subset_size = None
        run_ETL(subset_size)
    else:
        print("Bad input")
        sys.exit(0)
        

In [ ]:
block out placebo etc
retrieve/create cache
track progress of creation

In [21]:
test_or_prod()

The test run of this code performs the construction of the KG on a random subset of 200 Conditions, 200 Interventions, and 200 Alternate Interventions from Clinical Trials.



Is this a test run or the production of a new version of the KG? Enter Test for test, or Prod for production:  Test


Attempting download of Clinical Trial data as of 12_15_2023
Failed to scrape AACT for download. Please navigate to https://aact.ctti-clinicaltrials.org/download and manually download zip file.
Please store the downloaded zip in the /data directory


Type Done when done:  Done


File found at: 
/Users/Kamileh/Work/ISB/NCATS_BiomedicalTranslator/Projects/ClinicalTrials/ETL_Python/data/9opmph4n5l7055moqnfu3n6kxnc0.zip
Please make sure this the correct zip file from AACT
Unzipping data into
/Users/Kamileh/Work/ISB/NCATS_BiomedicalTranslator/Projects/ClinicalTrials/ETL_Python/data/12_11_2023_extracted
Approximate runtime for downloading or locating raw data: 0.0 hours, 1.0 minutes, 14.143150091171265 seconds
MetaMap version < 2020, conduct mapping on terms after removing ascii characters


% conditions mapped: 100%|██████████| 280/280 [03:12<00:00,  1.45it/s]


Using UMLS MetaMap to get mappings for INTERVENTIONS. MetaMap returns mappings, CUIs, and semantic type of mapping.
MetaMap version < 2020, conduct mapping on original interventions after removing ascii characters


% interventions mapped: 100%|███████| 293/293 [03:26<00:00,  1.42it/s]
% alternate_interventions mapped: 100%|█| 293/293 [03:26<00:00,  1.42i


Approximate runtime for mapping: 0.0 hours, 10.0 minutes, 7.185937881469727 seconds
Mapping UMLS CURIEs and names back to clinical trials
Scoring mappings
Auto-selecting high scoring CURIEs
Approximate runtime for overall mapping: 0.0 hours, 11.0 minutes, 27.955234050750732 seconds


In [ ]:
start_time_begin = time.time()
# flag_and_path = get_raw_ct_data() # download raw data
# end_time_download = time.time()
# elapsed_time = end_time_download - start_time_begin
# hours, minutes, seconds = convert_seconds_to_hms(elapsed_time)
# print(f"Approximate runtime for downloading or locating raw data: {hours} hours, {minutes} minutes, {seconds} seconds")
# flag_and_path = get_raw_ct_data() # download raw data


flag_and_path = {'term_program_flag': False,
                 # 'data_extracted_path': '/Users/Kamileh/Work/ISB/NCATS_BiomedicalTranslator/Projects/ClinicalTrials/ETL_Python/data/09_26_2023_extracted',
                 'data_extracted_path': '/Users/Kamileh/Work/ISB/NCATS_BiomedicalTranslator/Projects/ClinicalTrials/ETL_Python/data/12_11_2023_extracted',
                 'date_string':'12_11_2023'} # comment for production
subset_size = 20

global metamap_dirs
metamap_dirs = check_os()
df_dict = read_raw_ct_data(flag_and_path, subset_size) # read the clinical trial data

# term_list_to_cache()

start_time_mm = time.time()
term_list_to_mm(df_dict, flag_and_path) # map using MetaMap
end_time_mm = time.time()
elapsed_time = end_time_mm - start_time_mm
hours, minutes, seconds = convert_seconds_to_hms(elapsed_time)
print(f"Approximate runtime for mapping: {hours} hours, {minutes} minutes, {seconds} seconds")

map_to_trial(df_dict, flag_and_path) # map MetaMap terms back to trial 
score_mappings(flag_and_path) # score the mappings
auto_select_curies(flag_and_path) # select CURIEs automatically that pass score threshold
end_time_end = time.time()
elapsed_time = end_time_end - start_time_begin
hours, minutes, seconds = convert_seconds_to_hms(elapsed_time)
print(f"Approximate runtime for overall mapping: {hours} hours, {minutes} minutes, {seconds} seconds")

In [ ]:
# def run_ETL_mapping(flag_and_path):
#     df_dict = read_raw_ct_data(flag_and_path)
#     ct_terms = exact_match_mesh(df_dict)
#     ct_terms = inexact_match_mesh(df_dict, ct_terms)

#     # pull the available MeSH terms per study out of the returned ct_terms dict 
#     mesh_conditions_per_study = ct_terms["mesh_conditions_per_study"]
#     mesh_interventions_per_study = ct_terms["mesh_interventions_per_study"]

#     ct_terms = term_list_to_nr(df_dict, ct_terms)
#     ct_terms = term_list_to_mm(df_dict, ct_terms)

#     # pull the available UMLS terms per study out of the returned ct_terms dict 
#     all_metamapped_conditions = ct_terms["all_metamapped_conditions"]
#     all_metamapped_interventions = ct_terms["all_metamapped_interventions"]

#     remaining_unmapped_possible = {"mesh_conditions_per_study": mesh_conditions_per_study,
#                                    "mesh_interventions_per_study": mesh_interventions_per_study,
#                                    "all_metamapped_conditions": all_metamapped_conditions,
#                                    "all_metamapped_interventions": all_metamapped_interventions}
#     compile_and_output(df_dict, ct_terms, remaining_unmapped_possible)


    

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):  # more options can be specified also
    display(interventions_manual_review[:20])

In [ ]:
# flag_and_path = get_raw_ct_data() # uncomment for production
flag_and_path = {'term_program_flag': False,
                 # 'data_extracted_path': '/Users/Kamileh/Work/ISB/NCATS_BiomedicalTranslator/Projects/ClinicalTrials/ETL_Python/data/09_26_2023_extracted',
                 'data_extracted_path': '/Users/Kamileh/Work/ISB/NCATS_BiomedicalTranslator/Projects/ClinicalTrials/ETL_Python/data/9opmph4n5l7055moqnfu3n6kxnc0.zip',

                 'date_string':'12_11_2023'} # comment for production
metamap_dirs = check_os()
df_dict = read_raw_ct_data(flag_and_path, subset_size=100)
# term_list_to_mm(df_dict, flag_and_path)
# map_to_trial(df_dict, flag_and_path)
# score_mappings(flag_and_path)
# auto_select_curies(flag_and_path)





In [ ]:
def report_stats(df_dict, flag_and_path):
    """ Report counts of conditions, interventions"""
    relevant_date = flag_and_path["date_string"] # get date
    
    total_conditions = df_dict["conditions"].downcase_name
    total_conditions = list(total_conditions.unique())
    total_conditions = list(filter(None, total_conditions))
    
    orig_interventions = df_dict["interventions"]
    orig_interventions = orig_interventions['name'].str.lower()
    orig_interventions = list(orig_interventions.unique())
    orig_interventions = list(filter(None, orig_interventions))
    
    alt_interventions = df_dict["interventions_alts"].alt_downcase_name
    alt_interventions = list(alt_interventions.unique())
    alt_interventions = list(filter(None, alt_interventions))
    
#     metamap_input = "{}_metamap_output.tsv".format(relevant_date)
    
#     """ Get the full names of the semantic types and replace the abbreviations with the full names """
#     metamapped = pd.read_csv(metamap_input, sep='\t', index_col=False, header=0)

    print("Clinical Trial Data from: {}".format(relevant_date))
    print("Total # of unique conditions : {}".format(len(total_conditions)))
    print("Total # of unique interventions : {}".format(len(orig_interventions) + len(alt_interventions)))
    

    
